# A simple bijective network for data transformation using strings

A bijective data tranformation network is a series of nodes that represent different data formats with the same content. Nodes are connected by edges that are functions that are unidirectional mappings from one node to another. A graph is formed by showing that nodes that are transformed through a series of other nodes return to their original state. This requires that 
1. a node is well defined
2. a node has an equallity operator n1==n1'

The purpose of this network is to create a way of transforming data from one format to another as efficcently as possible while preserving contnent. As an example we use a series of data types that are strings.

In [8]:
# the edge from list of strings to string is defined in General Models
from pyMeasure.Code.DataHandlers.GeneralModels import *
# node 1
node_1="This is a test string\n it has to have multiple lines \n and many characters 34%6\n^"
# edge that maps node 1 to 2  
node_2=node_1.splitlines()
# edge that maps node 2 to 1
node_1_prime=string_list_collapse(node_2)
# make sure the round trip has not changed anything 
print("The assertion that n1==n1' is {0}").format(node_1==node_1_prime)
# we can add a file that is another node
# edge that maps node 1 to node 3
node_3=open("node_3.txt","w")
node_3.write(node_1)
node_3.close()
# now we can add an edge that maps node_3 to node_2
node_2_prime=[]
node_3=open("node_3.txt","r")
for line in node_3:
    node_2_prime.append(line.replace("\n",""))
node_3.close()
print("The assertion that n2==n2' is {0}").format(node_2==node_2_prime)

# now we can add a fourth node using the edge that maps node 2 to 4
node_4=[line+"\n" for line in node_2]
node_4[-1]=node_4[-1].replace("\n","")
node_1_double_prime=string_list_collapse(node_4,string_delimiter="")

print("The assertion that n1==n1'' is {0}").format(node_1==node_1_double_prime)

The assertion that n1==n1' is True
The assertion that n2==n2' is True
The assertion that n1==n1'' is True


## Now we define the graph class

In [61]:
import re
import datetime
class Graph():
    def __init__(self):
        self.node_names=['n1','n2']
        self.current_node='n1'
        self.state=[1,0]
        self.edges=[]
        self.data="This is a test string\n it has to have multiple lines \n and many characters 34%6\n^"
        
    def add_edge(self,begin_node=None,end_node=None,edge_function=None):
        """Adds an edge mapping one node to another"""
        # check to see if edge is defined if it is increment a number
        edge_match=re.compile("edge_{0}_{1}".format(begin_node,end_node))
        keys=self.__dict__.keys()
        #print keys
        iterator=0
        for key in keys:
            if re.match(edge_match,key):
                iterator+=1
        edge_name="edge_{0}_{1}_{2:0>3d}".format(begin_node,end_node,iterator)
        self.__dict__[edge_name]=edge_function
        self.edges.append(edge_name)
        
    def move_to(self,*path):
        
        for index,edge in enumerate(path):
            print self.data
            self.data=self.__dict__[edge](self.data)
            print self.data
            edge_pattern='edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)'
            match=re.match(edge_pattern,edge)
            self.current_node=match.groupdict()['end_node']
            self.state=[0 for i in range(len(self.node_names))]
            position=self.node_names.index(self.current_node)
            self.state[position]=1
            #print self.state
            print self.current_node
        
    
    def __str__(self):
        return str(self.data)
    
    def add_node(self,node_name,edge_into_node_begin,edge_into_node_function,edge_out_node_end,edge_out_node_function):
        # first check if node into and out of node is good
        self.node_names.append(node_name)
        self.state.append(0)
        self.add_edge(begin_node=node_name,end_node=edge_out_node_end,edge_function=edge_out_node_function)
        self.add_edge(begin_node=edge_into_node_begin,end_node=node_name,edge_function=edge_into_node_function)
    
    def path_length(self,num_repeats=10,*path):
        """Determines the length of a given path edge and saves it in self.edge_lenths"""
        begin_time=datetime.datetime.now()
        num_repeats=1
        for i in range(num_repeats):
            self.move_to(*path)
        end_time=datetime.datetime.now()
        return str(end_time-begin_time)
        
        
def edge_1_to_2(in_string):
    return in_string.splitlines()
    
def edge_2_to_1(string_list):
    return string_list_collapse(string_list)

def edge_1_to_3(in_string):
    node_3=open("node_3.txt","w")
    node_3.write(in_string)
    node_3.close()
    return "node_3.txt"
    
def edge_3_to_2(file_name):
    node_2_prime=[]
    node_3=open(file_name,"r")
    for line in node_3:
        node_2_prime.append(line.replace("\n",""))
    node_3.close()
    return node_2_prime
    
new_graph=Graph()
new_graph.add_edge('n1','n2',edge_1_to_2)
new_graph.add_edge('n2','n1',edge_2_to_1)
#print new_graph.__dict__
#new_graph.move_to(*new_graph.edges)
#print new_graph.edge_n1_n2_000(node_1)
#print new_graph.__dict__
new_graph.add_node('n3','n1',edge_1_to_3,'n2',edge_3_to_2)
print new_graph.edges
path=['edge_n1_n2_000', 'edge_n2_n1_000', 'edge_n1_n3_000', 'edge_n3_n2_000']
print new_graph.path_length(*path)
print new_graph.__dict__

['edge_n1_n2_000', 'edge_n2_n1_000', 'edge_n3_n2_000', 'edge_n1_n3_000']
This is a test string
 it has to have multiple lines 
 and many characters 34%6
^
This is a test string
 it has to have multiple lines 
 and many characters 34%6
^ was not <type 'list'>
T
h
i
s
 
i
s
 
a
 
t
e
s
t
 
s
t
r
i
n
g


 
i
t
 
h
a
s
 
t
o
 
h
a
v
e
 
m
u
l
t
i
p
l
e
 
l
i
n
e
s
 


 
a
n
d
 
m
a
n
y
 
c
h
a
r
a
c
t
e
r
s
 
3
4
%
6


^
n1
T
h
i
s
 
i
s
 
a
 
t
e
s
t
 
s
t
r
i
n
g


 
i
t
 
h
a
s
 
t
o
 
h
a
v
e
 
m
u
l
t
i
p
l
e
 
l
i
n
e
s
 


 
a
n
d
 
m
a
n
y
 
c
h
a
r
a
c
t
e
r
s
 
3
4
%
6


^
node_3.txt
n3
node_3.txt
['T', 'h', 'i', 's', ' ', 'i', 's', ' ', 'a', ' ', 't', 'e', 's', 't', ' ', 's', 't', 'r', 'i', 'n', 'g', '', '', ' ', 'i', 't', ' ', 'h', 'a', 's', ' ', 't', 'o', ' ', 'h', 'a', 'v', 'e', ' ', 'm', 'u', 'l', 't', 'i', 'p', 'l', 'e', ' ', 'l', 'i', 'n', 'e', 's', ' ', '', '', ' ', 'a', 'n', 'd', ' ', 'm', 'a', 'n', 'y', ' ', 'c', 'h', 'a', 'r', 'a', 'c', 't', 'e', 'r', 's', ' ', '3', '4

In [62]:
edge_1_to_2("my Test\n of this function")

['my Test', ' of this function']